In [86]:
import openmeteo_requests

import requests_cache
import pandas as pd
import numpy as np
from retry_requests import retry

from datetime import datetime

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": [21.0278, 16.047079, 10.762622],
	"longitude": [105.8342, 108.20623, 106.660172],
	"current": "temperature_2m",
	"daily": ["temperature_2m_max", "temperature_2m_min", "uv_index_max", "rain_sum", "precipitation_hours", "wind_speed_10m_max"],
	"timezone": "Asia/Bangkok",
	"past_days": 14,
	"forecast_days": 16
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
def daily(x):
    response = responses[x]

    daily = response.Daily()
    daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
    daily_uv_index_max = daily.Variables(2).ValuesAsNumpy()
    daily_rain_sum = daily.Variables(3).ValuesAsNumpy()
    daily_precipitation_hours = daily.Variables(4).ValuesAsNumpy()
    daily_wind_speed_10m_max = daily.Variables(5).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left")}
    
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    daily_data["uv_index_max"] = daily_uv_index_max
    daily_data["rain_sum"] = daily_rain_sum
    daily_data["precipitation_hours"] = daily_precipitation_hours
    daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max

    

    if x==0:
        daily_data["City"] = "Hà Nội"
    elif x==2:
        daily_data["City"] = "Đà Nẵng"
    else:
        daily_data["City"] = "TP Hồ Chí Minh"

    daily_dataframe = pd.DataFrame(data = daily_data)
    return daily_dataframe


def temp_current(x):
    response = responses[x]



    current = response.Current()
    current_temperature_2m = current.Variables(0).Value()
    return current_temperature_2m

dfHN = daily(0)

dfHN['Temp_curr'] = temp_current(0)

dfDN = daily(1)

dfDN['Temp_curr'] = temp_current(1)

dfHCM = daily(2)

dfHCM['Temp_curr'] = temp_current(2)

df = pd.concat([dfHN,dfDN,dfHCM],ignore_index=True)

df['Date'] = pd.to_datetime(df['date']).dt.date
df['Temperature_max']  =  round(df["temperature_2m_max"].astype(np.float64),2)
df['Temperature_min']  =  round(df["temperature_2m_min"].astype(np.float64),2)
df['Uv_max']  =  round(df["uv_index_max"].astype(np.float64),2)
df['Amount_rain']  =  round(df["rain_sum"].astype(np.float64),2)
df['Precipitation_rain']  =  round(df["precipitation_hours"].astype(np.float64),2)
df['Wind_max']  =  round(df["wind_speed_10m_max"].astype(np.float64),2)
df['Temp_curr2'] = round(df["Temp_curr"].astype(np.float64),2)


df.to_csv(r'C:\Users\Admin\Documents\WeatherApp\edit.csv')

